In [60]:
from scipy.spatial import distance
from imutils import face_utils
import imutils
import dlib
import cv2

In [61]:
import os
!git clone https://github.com/ntasfi/PyGame-Learning-Environment.git
os.chdir('PyGame-Learning-Environment')
!pip install -e .

fatal: destination path 'PyGame-Learning-Environment' already exists and is not an empty directory.
Obtaining file:///content/PyGame-Learning-Environment
  Found existing installation: ple 0.0.1
    Can't uninstall 'ple'. No files were found to uninstall.
  Running setup.py develop for ple


In [62]:
# Switch back to the root.
os.chdir('/content')

In [63]:
!pip install pygame

In [64]:
from pygame import mixer
def eye_aspect_ratio(eye):
	A = distance.euclidean(eye[1], eye[5])
	B = distance.euclidean(eye[2], eye[4])
	C = distance.euclidean(eye[0], eye[3])
	ear = (A + B) / (2.0 * C)
	return ear

In [65]:
thresh = 0.25
frame_check = 20
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")# Dat file is the crux of the code

In [59]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
cap=cv2.VideoCapture(0)
flag=0
while (True):
    ret, frame=cap.read()
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subjects = detect(gray, 0)
    for subject in subjects:
        shape=predict(gray,subject)
        shape=face_utils.shape_to_np(shape)
        leftEye=shape[lStart:lEnd]
        rightEye=shape[rStart:rEnd]
        leftEAR=eye_aspect_ratio(leftEye)
        rightEAR=eye_aspect_ratio(rightEye)
        ear=(leftEAR+rightEAR)/2.0
        leftEyeHull=cv2.convexHull(leftEye)
        rightEyeHull=cv2.convexHull(rightEye)
        cv2.drawContours(frame,[leftEyeHull],-1,(0,255,0),1)
        cv2.drawContours(frame,[rightEyeHull],-1,(0,255,0),1)
        if ear<thresh:
            flag+=1
            print(flag)
            
            if flag>=frame_check:
                cv2.putText(frame,"@@@@@@@@@@@WAKE UP BUDDY@@@@@@@@@@",(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
                cv2.putText(frame,"@@@@@@@@@@@WAKE UP BUDDY@@@@@@@@@@",(10,325),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
                mixer.init() #you must initialize the mixer
                alert=mixer.Sound('button-6.wav')
                alert.play()
                print("drowsy")
        else:
            flag=0
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            cap.release()
            break
cv2.destroyAllWindows()
cv2.stop()